In [1]:
%load_ext autoreload
%autoreload 2 
%reload_ext autoreload
%matplotlib inline
import matplotlib.pyplot as plt

# always import gbm_algos first !
import xgboost, lightgbm, catboost
from gplearn.genetic import SymbolicRegressor

# To access the contents of the parent dir
import sys; sys.path.insert(0, '../')
import os
from scipy.io import loadmat
from utils import *
from preprocess import *
from models import RobustPCANN

# Let's do facy optimizers
from optimizers import Lookahead, AdamGC, SGDGC
from madgrad import MADGRAD
from lbfgsnew import LBFGSNew

from pytorch_robust_pca import *

# Modify at /usr/local/lib/python3.9/site-packages/torch_lr_finder/lr_finder.py
from torch_lr_finder import LRFinder

# Tracking
from tqdm import trange

import sympy
import sympytorch

from pde_diff import *

Running Python 3.9.7
You can use npar for np.array


In [2]:
# Loading the KS sol
DATA_PATH = "../deephpms_data/KS_new2.pkl"
data = pickle_load(DATA_PATH)
t = data['t']
x = data['x']
X, T = np.meshgrid(x, t)
Exact = data['u'].T

# Adding noise
noise_intensity = 0.0
noisy_xt = False

if noise_intensity > 0.0:
    Exact = perturb(Exact, intensity=noise_intensity, noise_type="normal")
    print("Perturbed Exact with intensity =", float(noise_intensity))

x_star = X.T.flatten()[:, None]
t_star = T.T.flatten()[:, None]
X_star = np.hstack((x_star, t_star))
u_star = Exact.flatten()[:, None]

if noisy_xt: 
    print("Noisy (x, t)")
    X_star = perturb(X_star, intensity=noise_intensity, noise_type="normal")
else: print("Clean (x, t)")

# Doman bounds
lb = X_star.min(axis=0)
ub = X_star.max(axis=0)

print(f"Total data points {u_star.shape[0]}")
N = min(40000, u_star.shape[0])
print(f"Fine-tuning with {N} samples")
idx = np.random.choice(u_star.shape[0], N, replace=False)
X_u_train = X_star[idx, :]
u_train = u_star[idx,:]

# Convert to torch.tensor
X_u_train = to_tensor(X_u_train, True)
u_train = to_tensor(u_train, False)

# lb and ub are used in adversarial training
scaling_factor = 1.0
lb = scaling_factor*to_tensor(lb, False)
ub = scaling_factor*to_tensor(ub, False)

# Feature names, base on the symbolic regression results
feature_names=('uf', 'u_x', 'u_xx', 'u_xxxx'); feature2index = {}

Loaded from ../deephpms_data/KS_new2.pkl
Clean (x, t)
Total data points 29696
Fine-tuning with 29696 samples


In [3]:
program = '''
-1.048322*u_xx-1.055897*u_xxxx-0.997619*uf*u_x
'''

program = '''
-0.41179*u_xx-0.348230*u_xxxx-0.332343*uf*u_x
'''

program = '''
-1.007216*u_xx-1.0096300*u_xxxx-1.014672*uf*u_x
''' 

pde_expr, variables = build_exp(program); print(pde_expr, variables)
mod = sympytorch.SymPyModule(expressions=[pde_expr]); mod.train()

-1.014672*u_x*uf - 1.007216*u_xx - 1.00963*u_xxxx {u_xxxx, u_x, u_xx, uf}


SymPyModule(expressions=(-1.014672*u_x*uf - 1.007216*u_xx - 1.00963*u_xxxx,))

In [4]:
class RobustPINN(nn.Module):
    def __init__(self, model, loss_fn, index2features, scale=False, lb=None, ub=None, pretrained=False, noiseless_mode=True, init_cs=(0.5, 0.5), init_betas=(0.0, 0.0)):
        super(RobustPINN, self).__init__()
        self.model = model
        if not pretrained: self.model.apply(self.xavier_init)
        
        self.noiseless_mode = noiseless_mode
        if self.noiseless_mode: print("No denoising")
        else: print("With denoising method")
        
        self.in_fft_nn = None; self.out_fft_nn = None
        self.inp_rpca = None; self.out_rpca = None
        if not self.noiseless_mode:
            # FFTNN
            self.in_fft_nn = FFTTh(c=init_cs[0])
            self.out_fft_nn = FFTTh(c=init_cs[1])

            # Robust Beta-PCA
            self.inp_rpca = RobustPCANN(beta=init_betas[0], is_beta_trainable=True, inp_dims=2, hidden_dims=32)
            self.out_rpca = RobustPCANN(beta=init_betas[1], is_beta_trainable=True, inp_dims=1, hidden_dims=32)
        
        self.callable_loss_fn = loss_fn
        self.init_parameters = [nn.Parameter(torch.tensor(x.item())) for x in loss_fn.parameters()]
        self.param0 = self.init_parameters[0]
        self.param1 = self.init_parameters[1]
        self.param2 = self.init_parameters[2]
        del self.callable_loss_fn, self.init_parameters
        
        self.index2features = index2features; self.feature2index = {}
        for idx, fn in enumerate(self.index2features): self.feature2index[fn] = str(idx)
        self.scale = scale; self.lb, self.ub = lb, ub
        self.diff_flag = diff_flag(self.index2features)
        
    def xavier_init(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
        
    def forward(self, x, t):
        H = torch.cat([x, t], dim=1)
        if self.scale: H = self.neural_net_scale(H)
        return self.model(H)
    
    def loss(self, X_input, X_input_noise, y_input, y_input_noise, update_network_params=True, update_pde_params=True):
        # Denoising process
        if not self.noiseless_mode:
            # (1) Denoising FFT on (x, t)
            # This line returns the approx. recon.
            X_input_noise = cat(torch.fft.ifft(self.in_fft_nn(X_input_noise[1])*X_input_noise[0]).real.reshape(-1, 1), 
                                torch.fft.ifft(self.in_fft_nn(X_input_noise[3])*X_input_noise[2]).real.reshape(-1, 1))
            X_input_noise = X_input-X_input_noise
            X_input = self.inp_rpca(X_input, X_input_noise, normalize=True)
            
            # (2) Denoising FFT on y_input
            y_input_noise = y_input-torch.fft.ifft(self.out_fft_nn(y_input_noise[1])*y_input_noise[0]).real.reshape(-1, 1)
            y_input = self.out_rpca(y_input, y_input_noise, normalize=True)
        
        grads_dict, u_t = self.grads_dict(X_input[:, 0:1], X_input[:, 1:2])
        
        total_loss = []
        # MSE Loss
        if update_network_params:
            mse_loss = F.mse_loss(grads_dict["uf"], y_input)
            total_loss.append(mse_loss)
            
        # PDE Loss
        if update_pde_params:
            u_t_pred = (self.param0*grads_dict["uf"]*grads_dict["u_x"])+(self.param1*grads_dict["u_xx"])+(self.param2*grads_dict["u_xxxx"])
            l_eq = F.mse_loss(u_t_pred, u_t)
            total_loss.append(l_eq)
            
        return total_loss
    
    def grads_dict(self, x, t):
        uf = self.forward(x, t)
        u_t = self.gradients(uf, t)[0]
        u_x = self.gradients(uf, x)[0]
        u_xx = self.gradients(u_x, x)[0]
        u_xxx = self.gradients(u_xx, x)[0]
        u_xxxx = self.gradients(u_xxx, x)[0]        
        return {"uf":uf, "u_x":u_x, "u_xx":u_xx, "u_xxxx":u_xxxx}, u_t
    
    def get_selector_data(self, x, t):
        uf = self.forward(x, t)
        u_t = self.gradients(uf, t)[0]
        
        ### PDE Loss calculation ###
        # 'uf', 'u_x', 'u_xx', 'u_xxxx', 'u_xxx'
        u_x = self.gradients(uf, x)[0]
        u_xx = self.gradients(u_x, x)[0]
        u_xxx = self.gradients(u_xx, x)[0]
        u_xxxx = self.gradients(u_xxx, x)[0]
        u_xxxxx = self.gradients(u_xxxx, x)[0]
        derivatives = []
        derivatives.append(uf)
        derivatives.append(u_x)
        derivatives.append(u_xx)
        derivatives.append(u_xxx)
        derivatives.append(u_xxxx)
        derivatives.append(u_xxxxx)
        
        return torch.cat(derivatives, dim=1), u_t
    
    def gradients(self, func, x):
        return grad(func, x, create_graph=True, retain_graph=True, grad_outputs=torch.ones(func.shape))
    
    def neural_net_scale(self, inp): 
        return -1.0+2.0*(inp-self.lb)/(self.ub-self.lb)

In [5]:
model = TorchMLP(dimensions=[2, 50, 50, 50 ,50, 50, 1], activation_function=nn.Tanh, bn=nn.LayerNorm, dropout=None)
pinn = RobustPINN(model=model, loss_fn=mod, 
                  index2features=feature_names, scale=True, lb=lb, ub=ub, 
                  pretrained=True, noiseless_mode=True)
pinn = load_weights(pinn, "./new_saved_path/KS_new2_pretrained_pinn.pth")

# assigning the prefered loss_fn
model = pinn.model
pinn = RobustPINN(model=model, loss_fn=mod, 
                  index2features=feature_names, scale=True, lb=lb, ub=ub, 
                  pretrained=True, noiseless_mode=True)

Using old implementation of TorchMLP. See models.py for more new model-related source code.
No denoising
Loaded the model's weights properly
No denoising


#### Use STRidge to discover the hidden relation (on top of the pretrained solver net)

In [6]:
xx, tt = X_u_train[:, 0:1], X_u_train[:, 1:2]

pretraining_optimizer = LBFGSNew(pinn.model.parameters(),
                                 lr=1e-1, max_iter=500,
                                 max_eval=int(500*1.25), history_size=300,
                                 line_search_fn=True, batch_mode=False)

model.train()
for i in range(5):
    def pretraining_closure():
        global xx, tt, u_train
        if torch.is_grad_enabled(): pretraining_optimizer.zero_grad()
        mse_loss = F.mse_loss(pinn(xx, tt), u_train)
        if mse_loss.requires_grad: mse_loss.backward(retain_graph=False)
        return mse_loss

    pretraining_optimizer.step(pretraining_closure)

    if (i%1)==0:
        l = pretraining_closure()
        curr_loss = l.item()
        print("Epoch {}: ".format(i), curr_loss)

Epoch 0:  1.4179110621626023e-05
Epoch 1:  1.417903695255518e-05
Epoch 2:  1.4178962373989634e-05
Epoch 3:  1.4178842320689e-05
Epoch 4:  1.417844759998843e-05


In [7]:
inn = u_star
Ut, R, rhs_des = build_linear_system(inn.reshape(x.shape[0], t.shape[0]), t[1]-t[0], x[2]-x[1], D=5, P=5, time_diff = 'FD', space_diff = 'FD')
w = TrainSTRidge(R,Ut,10**-6,50)
print("PDE derived using STRidge")
print_pde(w, rhs_des)

PDE derived using STRidge
u_t = (-1.007216 +0.000000i)uu_{x}
    + (-1.009630 +0.000000i)u_{xx}
    + (-1.014672 +0.000000i)u_{xxxx}
   


In [8]:
# # xx, tt = X_u_train[:, 0:1], X_u_train[:, 1:2]

# NUMBER = 30000
# xx, tt = X_star[:NUMBER, 0:1], X_star[:NUMBER, 1:2]
# xx, tt = torch.FloatTensor(xx).requires_grad_(True), torch.FloatTensor(tt).requires_grad_(True)

# uf = pinn(xx, tt)
# u_t = pinn.gradients(uf, tt)[0]

# ### PDE Loss calculation ###
# # 'uf', 'u_x', 'u_xx', 'u_xxxx', 'u_xxx'
# u_x = pinn.gradients(uf, xx)[0]
# u_xx = pinn.gradients(u_x, xx)[0]
# u_xxx = pinn.gradients(u_xx, xx)[0]
# u_xxxx = pinn.gradients(u_xxx, xx)[0]
# # u_xxxxx = pinn.gradients(u_xxxx, xx)[0]

# derivatives = []
# derivatives.append(uf)
# derivatives.append(u_x)
# derivatives.append(u_xx)
# derivatives.append(u_xxx)
# derivatives.append(u_xxxx)
# # derivatives.append(u_xxxxx)
# derivatives = torch.cat(derivatives, dim=1)

# derivatives = derivatives.detach().numpy()
# u_t = u_t.detach().numpy()

In [9]:
# feature_names = ["uf", "u_x", "u_xx", "u_xxx", "u_xxxx"]

# X_input = derivatives
# y_input = u_t

# poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
# X_input = poly.fit_transform(X_input)

# poly_feature_names = poly.get_feature_names(feature_names)
# for i, f in enumerate(poly_feature_names): poly_feature_names[i] = f.replace(" ", "*")

In [10]:
# # # Set normalize=1
# w = TrainSTRidge(X_input[:, :], y_input, 1e-2, 100, l0_penalty=1, normalize=1)
# print("PDE derived using STRidge")
# print_pde(w, poly_feature_names[:])

In [11]:
_, x_fft, x_PSD = fft1d_denoise(X_u_train[:, 0:1], c=-5, return_real=True)
_, t_fft, t_PSD = fft1d_denoise(X_u_train[:, 1:2], c=-5, return_real=True)
_, u_train_fft, u_train_PSD = fft1d_denoise(u_train, c=-5, return_real=True)

In [12]:
x_fft, x_PSD = x_fft.detach(), x_PSD.detach()
t_fft, t_PSD = t_fft.detach(), t_PSD.detach()

In [13]:
def closure():
    if torch.is_grad_enabled():
        optimizer2.zero_grad()
    losses = pinn.loss(X_u_train, (x_fft, x_PSD, t_fft, t_PSD), u_train, (u_train_fft, u_train_PSD), update_network_params=True, update_pde_params=True)
    l = sum(losses)
    if l.requires_grad:
        l.backward(retain_graph=True)
    return l

def mtl_closure():
    losses = pinn.loss(X_u_train, (x_fft, x_PSD, t_fft, t_PSD), u_train, (u_train_fft, u_train_PSD), update_network_params=True, update_pde_params=True)
    updated_grads = []
    
    for i in range(len(losses)):
        optimizer1.zero_grad()
        losses[i].backward(retain_graph=True)

        g_task = []
        for param in pinn.parameters():
            if param.grad is not None:
                g_task.append(Variable(param.grad.clone(), requires_grad=False))
            else:
                g_task.append(Variable(torch.zeros(param.shape), requires_grad=False))
        # appending the gradients from each task
        updated_grads.append(g_task)

    updated_grads = list(pcgrad.pc_grad_update(updated_grads))[0]
    for idx, param in enumerate(pinn.parameters()): 
        param.grad = updated_grads[0][idx]+updated_grads[1][idx]
        
    return sum(losses)

In [14]:
pinn.loss(X_u_train, (x_fft, x_PSD, t_fft, t_PSD), u_train, (u_train_fft, u_train_PSD), update_network_params=True, update_pde_params=True)

[tensor(1.4178e-05, grad_fn=<MseLossBackward0>),
 tensor(0.0428, grad_fn=<MseLossBackward0>)]

In [15]:
epochs1, epochs2 = 30, 10
# TODO: Save best state dict and training for more epochs.
optimizer1 = MADGRAD(pinn.parameters(), lr=1e-5, momentum=0.9)
pinn.train(); best_train_loss = 1e6

print('1st Phase optimization using Adam with PCGrad gradient modification')
for i in range(epochs1):
    optimizer1.step(mtl_closure)
    if (i % 10) == 0 or i == epochs1-1:
        l = mtl_closure()
        print("Epoch {}: ".format(i), l.item())
        print(pinn.param0, pinn.param1, pinn.param2)
        
optimizer2 = torch.optim.LBFGS(pinn.parameters(), lr=1e-1, max_iter=500, max_eval=int(500*1.25), history_size=300, line_search_fn='strong_wolfe')
print('2nd Phase optimization using LBFGS')
for i in range(epochs2):
    optimizer2.step(closure)
    if (i % 10) == 0 or i == epochs2-1:
        l = closure()
        print("Epoch {}: ".format(i), l.item())

1st Phase optimization using Adam with PCGrad gradient modification
Epoch 0:  0.04005521535873413
Parameter containing:
tensor(-1.0096, requires_grad=True) Parameter containing:
tensor(-1.0072, requires_grad=True) Parameter containing:
tensor(-1.0146, requires_grad=True)
Epoch 10:  0.016976967453956604
Parameter containing:
tensor(-1.0091, requires_grad=True) Parameter containing:
tensor(-1.0081, requires_grad=True) Parameter containing:
tensor(-1.0137, requires_grad=True)
Epoch 20:  0.01190744899213314
Parameter containing:
tensor(-1.0082, requires_grad=True) Parameter containing:
tensor(-1.0092, requires_grad=True) Parameter containing:
tensor(-1.0127, requires_grad=True)
Epoch 29:  0.00998506136238575
Parameter containing:
tensor(-1.0071, requires_grad=True) Parameter containing:
tensor(-1.0103, requires_grad=True) Parameter containing:
tensor(-1.0117, requires_grad=True)
2nd Phase optimization using LBFGS
Epoch 0:  0.0016022277995944023
Epoch 9:  0.0016022275667637587


In [16]:
pred_params = [pinn.param0.item(), pinn.param1.item(), pinn.param2.item()]
print(pred_params)

[-0.8170313239097595, -1.0996100902557373, -0.9884783029556274]


In [17]:
errs = 100*np.abs(np.array(pred_params)+1)
print(errs.mean(), errs.std())

9.803348779678345 7.0001813845422385


In [ ]:
### New results ###
# finetuned pinn

# w/o DFT
# clean all
# [-1.0038105249404907, -1.0389138460159302, -1.018654465675354]
# 2.0459612210591636 1.4387603533171842

# w/o DFT
# clean (x, t) and noisy lables
# 1st run
# [-0.9552096724510193, -1.0218358039855957, -1.0091873407363892]
# 2.527115742365519 1.4736449310642017
# 2nd run
# [-0.9824689626693726, -1.0259006023406982, -1.0224767923355103]
# 2.1969477335611978 0.34356397750920376

# w/o DFT
# noisy (x, t) and noisy lables
# [-0.9445462822914124, -1.0215760469436646, -1.0037590265274048]
# 2.692959705988566 2.1441092139699642

###############################
# w/ DFT
# clean all
# [-0.9454907774925232, -1.0266990661621094, -1.0051020383834839]
# 2.8770109017690024 2.0223490873202765

# w/ DFT
# clean (x, t) and noisy lables
# [-0.965919554233551, -1.0234534740447998, -1.0168591737747192]
# 2.479769786198934 0.7094516920660559

# w/ DFT
# noisy (x, t) and noisy lables
# [-0.9662449955940247, -1.0259292125701904, -1.0150245428085327]
# 2.4902919928232827 0.7681037488669546

In [ ]:
### New results ###

# w/o DFT
# noisy labels
# [-0.9948846697807312, -1.0065652132034302, -0.9950999617576599]
# 0.5526860555013021 0.07394681988603513

# noisy (x, t) and labels
# [-0.9892916083335876, -1.011358618736267, -0.9886858463287354]
# 1.1127054691314697 0.029659549537678777

# /w DFT
# clean | /w DFT
# [-0.9948897361755371, -1.0049610137939453, -0.9951441287994385]
# 0.4975716272989909 0.01043744028051521

# noisy (x, t) and labels | /w DFT
# [-0.9947568774223328, -1.0052711963653564, -0.9935149550437927]
# 0.5666454633076986 0.05789442242742208

In [ ]:
### Without AutoEncoder ###

# Clean Exact and (x, t)
# [-0.999634325504303, -0.9994997382164001, -0.9995566010475159]
# (0.04364450772603353, 0.005516461306387781)

# Pretrained with final_finetuned_pinn_5000 (do not use / report)
# [-0.9996052980422974, -0.9995099902153015, -0.9995319247245789]
# 0.04509290059407552 0.0040754479422416435

# Noisy Exact and clean (x, t)
# [-0.9967969655990601, -0.9969800114631653, -0.9973703026771545]
# (0.2950906753540039, 0.023910676954986623)

# Noisy Exact and noisy (x, t)
# [-0.9975059032440186, -0.9962050914764404, -0.9969104528427124]
# 0.3126184145609538 0.05316856937153804

In [ ]:
### Without AutoEncoder & With Robust PCA ###

# Noisy Exact and clean (x, t)
# [-1.0003160238265991, -1.0005097389221191, -0.9997726678848267]
# (0.035103162129720054, 0.011791963483533422)

# Noisy Exact and noisy (x, t) + Robust PCA
# [-1.000351071357727, -0.9991073608398438, -0.9980993866920471]
# (0.08140603701273601 0.09209241474722876)

In [ ]:
### Without AutoEncoder & With Double Beta-Robust PCA ###

# Noisy Exact and clean (x, t)
# [-1.0013889074325562, -0.998389720916748, -1.0010501146316528]
# 0.1349767049153646 0.023035484282371465

# Noisy Exact and noisy (x, t) + Robust PCA
# [-0.9980248212814331, -0.9982557892799377, -0.998869001865387]
# 0.1616795857747396 0.03562172791384541